In [ ]:
!pip install langchain qdrant-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown https://drive.google.com/file/d/10GJNHfEeqpjq5AWEVn7azs_Utrk5v3wh/view?usp=sharing -O corpus.txt

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=10GJNHfEeqpjq5AWEVn7azs_Utrk5v3wh
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/10GJNHfEeqpjq5AWEVn7azs_Utrk5v3wh/view?usp=sharing
To: /content/corpus.txt
82.8kB [00:00, 2.51MB/s]


In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader,TextLoader

loader_corpus = TextLoader("/content/drive/MyDrive/vi-medical-chatbot/corpus.txt")

In [ ]:
# Cắt pdf thành các đoạn text nhỏ, size là 1000 mỗi đoạn
documents_corpus = loader_corpus.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)

texts_corpus = text_splitter.split_documents(documents_corpus)
len(texts_corpus)


11916

In [ ]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = "<QDRANT_API_KEY>"
QDRANT_URL = "<QDRANT_URL>"
HUGGINGFACE_API_KEY= "<HUGGINGFACE_API_KEY>"

In [ ]:

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# Tạo lớp embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key = HUGGINGFACE_API_KEY,
    model_kwargs = {'device': 'auto'}
)

# Encode dữ liệu text qua lớp embedding,
# rồi lưu lại trên vector database server
qdrant = Qdrant.from_documents(
    texts_corpus,
    embeddings,
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="disease_corpus_vector_db_v1",
    api_key=QDRANT_API_KEY,
)

print("Tạo vector database thành công!")

Tạo vector database thành công!


In [ ]:
query = "Hiếm muộn là gì?"
found_docs = qdrant.similarity_search(query,k=5)

In [ ]:
found_docs

[Document(page_content='Hiếm muộn là gì? Hiếm muộn theo định nghĩa của Tổ chức Y tế Thế giới (WHO) là tình trạng một cặp vợ chồng không thể có thai sau một năm chung sống, giao hợp thường xuyên và không sử dụng bất kỳ biện pháp tránh thai nào. Nếu người vợ trên 35 tuổi, thời gian này được rút ngắn lại chỉ còn 6 tháng. (1) Theo các chuyên gia, khoảng 85% phụ nữ sẽ thụ thai tự nhiên trong vòng 12 tháng đầu tiên. Trong 3 tháng đầu có quan hệ tình dục không sử dụng bất kỳ biện pháp tránh thai nào thì khả năng sinh sản là 25%. Tỷ lệ này sẽ giảm xuống còn 15% trong 9 tháng còn lại. Vì vậy với những cặp vợ chồng đang mong con và đã cố gắng trong ít nhất một năm mà vẫn chưa có tin vui thì nên đi khám chuyên khoa càng sớm càng tốt, việc điều trị không nên trì hoãn vì càng lớn tuổi chức năng sinh sản sẽ càng suy giảm. Hiếm muộn có thể được phân thành hai loại gồm hiếm muộn nguyên phát và thứ phát. Trong đó, hiếm muộn thứ phát là trường hợp hai vợ chồng đã có thai ít nhất một lần. Hiếm muộn nguyê